In [15]:
from hyrax import Hyrax

h = Hyrax(config_file='./astrominn_testing_runtime_config.toml')
h.config['train']['epochs'] = 1

[2025-12-12 15:26:26,876 hyrax.config_utils:INFO] Merging external default config from /Users/drew/code/applecider/src/applecider/default_config.toml
[2025-12-12 15:26:26,887 hyrax.config_utils:WARNING] Runtime config contains key or section 'train' which has no default defined. All configuration keys and sections must be defined in /Users/drew/code/hyrax/src/hyrax/hyrax_default_config.toml
[2025-12-12 15:26:26,888 hyrax.config_utils:WARNING] Runtime config contains key or section 'infer' which has no default defined. All configuration keys and sections must be defined in /Users/drew/code/hyrax/src/hyrax/hyrax_default_config.toml


In [2]:
ds = h.prepare()

/Users/drew/opt/miniconda3/envs/applecider/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
[2025-12-12 15:12:14,148 hyrax.prepare:INFO] Finished Prepare


In [3]:
ds['train'][10_431]

{'data': {'image': tensor([[[ 0.0160,  0.0163,  0.0163,  ...,  0.0156,  0.0159,  0.0160],
           [ 0.0156,  0.0147,  0.0159,  ...,  0.0147,  0.0163,  0.0166],
           [ 0.0153,  0.0166,  0.0150,  ...,  0.0154,  0.0157,  0.0162],
           ...,
           [ 0.0154,  0.0158,  0.0158,  ...,  0.0157,  0.0149,  0.0149],
           [ 0.0163,  0.0163,  0.0156,  ...,  0.0160,  0.0159,  0.0160],
           [ 0.0153,  0.0159,  0.0166,  ...,  0.0160,  0.0156,  0.0155]],
  
          [[ 0.0159,  0.0156,  0.0155,  ...,  0.0160,  0.0159,  0.0158],
           [ 0.0159,  0.0157,  0.0156,  ...,  0.0158,  0.0159,  0.0158],
           [ 0.0159,  0.0157,  0.0159,  ...,  0.0158,  0.0158,  0.0158],
           ...,
           [ 0.0159,  0.0158,  0.0158,  ...,  0.0159,  0.0154,  0.0158],
           [ 0.0159,  0.0159,  0.0160,  ...,  0.0159,  0.0156,  0.0159],
           [ 0.0159,  0.0158,  0.0156,  ...,  0.0156,  0.0158,  0.0159]],
  
          [[ 0.0068,  0.0076,  0.0263,  ..., -0.0134, -0.0028,  0.0

In [4]:
h.set_config('model.AstroMiNN.use_probabilities', False)
m = h.train()

[2025-12-12 15:12:16,446 hyrax.config_utils:WARNING] Cannot find default_config.toml for torch.nn.CrossEntropyLoss.
[2025-12-12 15:12:16,446 hyrax.config_utils:WARNING] Cannot find default_config.toml for torch.optim.SGD.
[2025-12-12 15:12:19,294 hyrax.config_utils:WARNING] Cannot find default_config.toml for umap.UMAP.
[2025-12-12 15:12:19,294 hyrax.config_utils:INFO] Merging external default config from /Users/drew/code/applecider/src/applecider/default_config.toml
[2025-12-12 15:12:21,452 hyrax.models.model_registry:INFO] Setting model's self.optimizer from config: torch.optim.SGD with arguments: {'lr': 0.01, 'momentum': 0.9}.
[2025-12-12 15:12:21,453 hyrax.models.model_registry:INFO] Setting model's self.criterion from config: torch.nn.CrossEntropyLoss with default arguments.
[2025-12-12 15:12:21,454 hyrax.verbs.train:INFO] Training model: AstroMiNN
[2025-12-12 15:12:21,454 hyrax.verbs.train:INFO] Training dataset(s):
{'train': Name: data (primary dataset)
  Dataset class: applecid

  5%|4         | 1/21 [00:00<?, ?it/s]

[2025-12-12 15:13:07,524 hyrax.pytorch_ignite:INFO] Total training time: 45.58[s]
2025/12/12 15:13:07 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/12/12 15:13:07 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
[2025-12-12 15:13:07,860 hyrax.verbs.train:INFO] Finished Training


In [5]:
h.to_onnx()

[2025-12-12 15:13:13,048 hyrax.config_utils:WARNING] Cannot find default_config.toml for torch.nn.CrossEntropyLoss.
[2025-12-12 15:13:13,049 hyrax.config_utils:WARNING] Cannot find default_config.toml for torch.optim.SGD.
[2025-12-12 15:13:13,051 hyrax.config_utils:WARNING] Cannot find default_config.toml for umap.UMAP.
[2025-12-12 15:13:13,052 hyrax.config_utils:INFO] Merging external default config from /Users/drew/code/applecider/src/applecider/default_config.toml
[2025-12-12 15:13:13,071 hyrax.config_utils:WARNING] Runtime config contains key or section 'train' which has no default defined. All configuration keys and sections must be defined in /Users/drew/code/hyrax/src/hyrax/hyrax_default_config.toml
[2025-12-12 15:13:13,071 hyrax.config_utils:WARNING] Runtime config contains key or section 'infer' which has no default defined. All configuration keys and sections must be defined in /Users/drew/code/hyrax/src/hyrax/hyrax_default_config.toml
[2025-12-12 15:13:15,827 hyrax.models.mo

In [6]:
h.engine()

[2025-12-12 15:13:50,957 hyrax.data_sets.inference_dataset:WARNING] Shape of structured_batch: (512,), dtype: [('id', '<U12'), ('tensor', '<f4', (5,))]
[2025-12-12 15:13:55,011 hyrax.data_sets.inference_dataset:WARNING] Shape of structured_batch: (512,), dtype: [('id', '<U12'), ('tensor', '<f4', (5,))]
[2025-12-12 15:13:59,127 hyrax.data_sets.inference_dataset:WARNING] Shape of structured_batch: (512,), dtype: [('id', '<U12'), ('tensor', '<f4', (5,))]
[2025-12-12 15:14:03,199 hyrax.data_sets.inference_dataset:WARNING] Shape of structured_batch: (512,), dtype: [('id', '<U12'), ('tensor', '<f4', (5,))]
[2025-12-12 15:14:07,231 hyrax.data_sets.inference_dataset:WARNING] Shape of structured_batch: (512,), dtype: [('id', '<U12'), ('tensor', '<f4', (5,))]
[2025-12-12 15:14:11,241 hyrax.data_sets.inference_dataset:WARNING] Shape of structured_batch: (512,), dtype: [('id', '<U12'), ('tensor', '<f4', (5,))]
[2025-12-12 15:14:15,250 hyrax.data_sets.inference_dataset:WARNING] Shape of structured_

In [16]:
# h.set_config('model.AstroMiNN.use_probabilities', True)
infer_ds = h.infer()

[2025-12-12 15:26:32,904 hyrax.models.model_registry:INFO] Setting model's self.optimizer from config: torch.optim.SGD with arguments: {'lr': 0.01, 'momentum': 0.9}.
[2025-12-12 15:26:32,907 hyrax.models.model_registry:INFO] Setting model's self.criterion from config: torch.nn.CrossEntropyLoss with default arguments.
[2025-12-12 15:26:32,907 hyrax.verbs.infer:INFO] Inference model: AstroMiNN
[2025-12-12 15:26:32,908 hyrax.verbs.infer:INFO] Inference dataset(s):
{'train': Name: data (primary dataset)
  Dataset class: applecider.datasets.image_and_metadata_dataset.ImageAndMetadataDataset
  Data location: ../../../data/preprocessed_ztf_alerts
  Primary ID field: obj_id
  Requested fields: image, metadata, target, obj_id
, 'infer': Name: data (primary dataset)
  Dataset class: applecider.datasets.image_and_metadata_dataset.ImageAndMetadataDataset
  Data location: ../../../data/preprocessed_ztf_alerts
  Primary ID field: obj_id
  Requested fields: image, metadata, obj_id
}
2025-12-12 15:26:

  5%|4         | 1/21 [00:00<?, ?it/s]

[2025-12-12 15:26:36,039 hyrax.data_sets.inference_dataset:WARNING] Shape of structured_batch: (512,), dtype: [('id', '<U12'), ('tensor', '<f4', (5,))]
[2025-12-12 15:26:36,280 hyrax.data_sets.inference_dataset:WARNING] Shape of structured_batch: (512,), dtype: [('id', '<U12'), ('tensor', '<f4', (5,))]
[2025-12-12 15:26:36,531 hyrax.data_sets.inference_dataset:WARNING] Shape of structured_batch: (512,), dtype: [('id', '<U12'), ('tensor', '<f4', (5,))]
[2025-12-12 15:26:36,781 hyrax.data_sets.inference_dataset:WARNING] Shape of structured_batch: (512,), dtype: [('id', '<U12'), ('tensor', '<f4', (5,))]
[2025-12-12 15:26:37,019 hyrax.data_sets.inference_dataset:WARNING] Shape of structured_batch: (512,), dtype: [('id', '<U12'), ('tensor', '<f4', (5,))]
[2025-12-12 15:26:37,260 hyrax.data_sets.inference_dataset:WARNING] Shape of structured_batch: (512,), dtype: [('id', '<U12'), ('tensor', '<f4', (5,))]
[2025-12-12 15:26:37,505 hyrax.data_sets.inference_dataset:WARNING] Shape of structured_

In [ ]:
import numpy as np
res_list = []

for i in range(21):
    a = np.load(f"/Users/drew/code/applecider/docs/pre_executed/testing/results/20251121-093202-infer-MNMd/batch_{i}.npy")

    for i in a:
        max_ind = np.argmax(i['tensor'])
        res_list.append((str(i['id']), int(max_ind)))

In [ ]:
import numpy as np
import glob

CLASSES = [
    ['SN Ia','SN Ic','SN Ib'],
    ['SN IIP', 'SN IIn','SN II', 'SN IIb'],
    ['Cataclysmic'],
    ['AGN'],
    ['Tidal Disruption Event']
]

real_class = []
pred_class = []
for k, v in res_list:
    # get all files in data directory that start with k
    files = glob.glob(f"/Users/drew/code/applecider/data/preprocessed_ztf_alerts/{k}*.npy")
    b = np.load(files[0], allow_pickle=True)

    pred_class.append(v)

    target = b.item().get('target')

    for i, c in enumerate(CLASSES):
        if target in c:
            target_class = i
            break

    real_class.append(target_class)


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
ConfusionMatrixDisplay(confusion_matrix(real_class, pred_class)).plot()

In [17]:
import numpy as np

onnx = np.load('/Users/drew/code/applecider/docs/pre_executed/testing/results/20251212-151346-engine-pDMl/batch_0.npy', allow_pickle=True)
pyt = np.load('/Users/drew/code/applecider/docs/pre_executed/testing/results/20251212-152630-infer--fRb/batch_0.npy', allow_pickle=True)

In [18]:
print(onnx[0:5])
print(pyt[0:5])

[('ZTF24aabpwcz', [ 8.64001083e+00,  5.51859379e+00,  5.66601753e-03,  1.24483261e+01, -4.57018465e-01])
 ('ZTF24aalywdn', [ 1.14223137e+01,  6.71184874e+00,  2.98581266e+00,  9.52106094e+00,  5.41381478e-01])
 ('ZTF24aazmqbf', [ 1.11223459e+01,  6.57421494e+00,  3.17251205e+00,  9.55171204e+00,  4.47753727e-01])
 ('ZTF24aahxgoa', [ 1.13685083e+01,  6.20165634e+00,  2.93455219e+00,  9.56226444e+00,  4.85990465e-01])
 ('ZTF24aaqxajf', [ 1.11676655e+01,  6.28761721e+00,  3.01888704e+00,  9.58392143e+00,  3.73888075e-01])]
[('ZTF24aaoupov', [ 1.1154709e+01,  6.5582542e+00,  3.1636550e+00,  9.3121090e+00,  5.0841039e-01])
 ('ZTF24aazlori', [ 1.1491161e+01,  6.3558493e+00,  2.9785633e+00,  9.7362413e+00,  3.6069369e-01])
 ('ZTF24aabpwcz', [ 8.6400099e+00,  5.5185957e+00,  5.6650639e-03,  1.2448328e+01, -4.5701772e-01])
 ('ZTF24aazlori', [ 1.1428663e+01,  6.2792683e+00,  3.0198116e+00,  9.7767487e+00,  2.5963283e-01])
 ('ZTF24aaybafr', [ 1.0846497e+01,  6.3679738e+00,  2.8795891e+00,  9.6375